# Tuning

In [3]:
import numpy as np
import pysmurf

S = pysmurf.SmurfControl(cfg_file='')

ModuleNotFoundError: No module named 'pysmurf'

## Coarse Tuning

Coarse tuning is a quick search for resonator dips. We generate a known broad band noise signal out of the DAC. We then measure the response in the ADC. The cross correlation between these input and the output has notches where the resonators are. We refer to this process as full_band_resp. Since we know the phase of our input/output data, we have both amplitude and phase information. To do the coarse peak finding manually

In [ ]:
f, resp = S.full_band_resp(2)
peaks = S.find_peak(f, resp, make_plot=True, show_plot=True)

We take the cross correlation data and look for phase slips and resonator dips. The x's in the top panel show the found resonators. The first command returns the frequency data in Hz and the complex response. The second command finds the peaks. Both functions have optional variables that give more flexibility.

## Resonator location refinement

This coarse search is very good at finding the approximate location of all the resonators. However we need knowledge of the resonator locations in frequency to an accuracy of about 10 kHz. To further refines the peak search, a gradient descent minima search is run. This is run in pyrogue to speed up the process. This takes about 2 minutes for 150 resonators.

TODO: ADD THE CODE SNIPPET FOR TUNING!

The resonator tracking parameter is referred to as "eta". It is a single complex number that gives the phase and width of the resonator. We will begin to discuss things in terms of I and Q, the quadrature decomposition of the signal. I is the in phase component, and Q is the quadrature (90 deg out of phase) component. The eta parameter is designed to rotate the resonator response at the minima into Q.

In [ ]:
S.load_tune(last_tune=True)
S.plot_tune_summary(2, eta_scan=True, show_plot=False)

On the top left is the response: amplitude in color, I in small dashed, and Q in long dashed. Below that is the phase response. Note that the phase is steepest at the minima. This is why we use the gradient of the phase to look for resonator minima. On the right is the resonator response in I and Q. The goal of the eta parameter is to find the complex number that rotates and scales the resonator response so that all changes are along the Q-axis. This allows the tracking algorithm to only monitor Q, which is much faster.

# Tracking

Now that the resonators are rotated into Q-only, the system must track the tones. Input power onto the TES changes the characteristic impedance of the resonator circuit and shifts the resonances up and down in frequency. We monitor this change by coupling the resonators to an RF SQUID.

To avoid TLS noise, we shift the signal up in frequency by flux ramping the SQUIDs. SQUIDs have a periodic V-phi curve, so if you pass enough voltage across a SQUID, it will trace out an approximately sinusoidal response.

To run setup tracking on band 2


In [ ]:
S.tracking_setup(2, make_plot=True)

The flux ramp is a triangle wave. In the top panel is the tracked fluxed ramp response of the SQUID. The bottom panel is the residual, or "tracking error". The frequency swing in the tracked section is about 100 kHz. Typical tracking errors are below 10 kHz. We use the amplitude of the frequency swing and the standard deviation of the tracking error to turn off bad channels. To check the channels

In [ ]:
S.check_lock(2)

And to see the tracking parameters of the remaining resonators after checking lock run tracking_setup again

In [ ]:
S.tracking_setup(2, make_plot=True)